In [33]:
import pandas as pd
import numpy as np
import sklearn
from scipy.stats import shapiro
import statistics
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("BankChurners.csv")

In [16]:
df.head(2)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
0,768805383,1,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1144,42,0.061
1,818770008,1,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1291,33,0.105


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  int32  
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

In [5]:
## 
df["Attrition_Flag"] = LabelEncoder().fit_transform(df["Attrition_Flag"])

In [18]:
df_numericas = df.select_dtypes(include=['float64', 'int64','int32'])

In [19]:
y = df["Attrition_Flag"] # variable dependiente/target

In [20]:
x = df_numericas.drop(columns=["Attrition_Flag"])

In [24]:
# Usaremos mutual_info_classif (es apto para no normalidad)

selector = SelectKBest(score_func=mutual_info_classif, k=5)  # elige top 5
selector.fit(x, y)

scores = pd.DataFrame({
    "Variable": x.columns,
    "Score": selector.scores_
}).sort_values(by="Score", ascending=False)

print(scores)

                    Variable     Score
10           Total_Trans_Amt  0.158454
11            Total_Trans_Ct  0.107256
8        Total_Revolving_Bal  0.085597
12     Avg_Utilization_Ratio  0.049919
6      Contacts_Count_12_mon  0.026546
5     Months_Inactive_12_mon  0.022722
9            Avg_Open_To_Buy  0.021120
4   Total_Relationship_Count  0.008635
7               Credit_Limit  0.007987
2            Dependent_count  0.004864
0                  CLIENTNUM  0.004446
3             Months_on_book  0.000944
1               Customer_Age  0.000361


Se utilizó el método de selección de características SelectKBest con la métrica mutual_info_classif, dado que los datos no presentan normalidad. Este método evalúa la dependencia estadística entre cada variable y la variable objetivo. Los resultados muestran que las variables relacionadas con la cantidad y el monto de transacciones son las más relevantes para explicar la probabilidad de abandono del cliente (customer attrition)

 Dado que las variables numéricas no cumplen el supuesto de normalidad (p < 0.05 en Shapiro-Wilk), se utilizó el método SelectKBest con mutual_info_classif, apropiado para datos no paramétricos.

Adicionalmente, se aplicó RFE con regresión logística y un modelo Random Forest, los cuales no requieren normalidad estricta y permiten validar las características relevantes.

De esta forma, se seleccionaron las variables siguiendo los supuestos estadísticos de cada método.

In [30]:
modelo = LogisticRegression(max_iter=500)
rfe = RFE(modelo, n_features_to_select=5)
rfe.fit(x,y)

rfe_results = pd.DataFrame({
    "Variable": x.columns,
    "Selected": rfe.support_,
    "Ranking": rfe.ranking_
})


C:\Users\JosedeJesusMoralesRe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\JosedeJesusMoralesRe\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

In [31]:
print(rfe_results.sort_values("Ranking"))

                    Variable  Selected  Ranking
2            Dependent_count      True        1
4   Total_Relationship_Count      True        1
5     Months_Inactive_12_mon      True        1
6      Contacts_Count_12_mon      True        1
12     Avg_Utilization_Ratio      True        1
11            Total_Trans_Ct     False        2
3             Months_on_book     False        3
1               Customer_Age     False        4
8        Total_Revolving_Bal     False        5
10           Total_Trans_Amt     False        6
7               Credit_Limit     False        7
9            Avg_Open_To_Buy     False        8
0                  CLIENTNUM     False        9


Mutual Information destacó las variables de actividad transaccional, mientras que RFE seleccionó variables de engagement administrativo y uso del crédito.

In [34]:
rf = RandomForestClassifier()
rf.fit(x, y)

importances = pd.DataFrame({
    "Variable": x.columns,
    "Importance": rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

In [35]:
print(importances)

                    Variable  Importance
10           Total_Trans_Amt    0.232606
11            Total_Trans_Ct    0.197525
8        Total_Revolving_Bal    0.123532
4   Total_Relationship_Count    0.081105
12     Avg_Utilization_Ratio    0.071383
7               Credit_Limit    0.047490
1               Customer_Age    0.045374
9            Avg_Open_To_Buy    0.044505
0                  CLIENTNUM    0.040363
6      Contacts_Count_12_mon    0.035490
5     Months_Inactive_12_mon    0.031944
3             Months_on_book    0.031907
2            Dependent_count    0.016776


Los tres métodos de selección de características coinciden en que las variables relacionadas con la actividad financiera del cliente (Número total de transacciones 12m, Monto total de transacciones 12m, % de uso del crédito) y el nivel de relación con el banco (Número total de productos bancarios) son los predictores más relevantes del abandono.
Este hallazgo es consistente con el comportamiento esperado en productos crediticios, donde la disminución en el uso y el bajo engagement suelen anticipar el churn.

La información mutua es adecuada para detectar relaciones no lineales entre variables y la variable objetivo (Brown et al., 2012).

“Tree-based feature importance is robust to outliers and does not require linearity or feature scaling.” (Breiman, L. 2001)

## Referencias

Brown, G., Pocock, A., Zhao, M.-J., & Luján, M. (2012).  
*Conditional likelihood maximisation: A unifying framework for information theoretic feature selection.*  
Journal of Machine Learning Research, 13, 27–66.

Breiman, L. (2001).  
*Random forests.* Machine Learning, 45(1), 5-32.


Discute por qué crees que las características seleccionadas son las más relevantes y por qué el resto quedaron excluidas en la selección

Un cliente activo, endeudado y con más productos tiende a permanecer mientras un cliente inactivo y con poco vínculo tiende a irse

Por otro lado, las variables demográficas y de capacidad crediticia pueden correlacionar, pero no son tan fuertes como las conductuales.

Las variables elegidas tienen sentido desde tres dimensiones clave del comportamiento del cliente financiero uso, compromiso financiero y relación ocn el banco mientras que las variables excluidas tendieron a ser descriptivas y no conductuales ejemplo edad y dependientes.